In [2]:
import pandas as pd
import csv
from string import ascii_letters, punctuation
letters = ascii_letters + punctuation
from nltk.corpus import words, stopwords
import nltk
from collections import Counter, defaultdict
import re, hashlib, os
endict = set(words.words())
from IPython.core.debugger import Tracer; debug_here = Tracer()
from pandas.tseries.offsets import *
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn import svm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import make_pipeline
from scipy.stats import ttest_rel, ttest_ind, pearsonr
from scipy.sparse import hstack
import numpy as np
import math
import sys
csv.field_size_limit(sys.maxsize)

131072

# Filter cs_talk_scores.csv

In [2]:
# See if a word is all Latin characters
def all_latin(word):
    if len(word) > 1 and all(c in letters for c in word) \
            and any(c in ascii_letters for c in word) \
            and not word.startswith('http') \
            and word != '(UTC)' \
            and word != '\\n' \
            and not 'TEMPLATE' in word:
        return True
    else:
        return False

# See if a text has at least three words with Latin letters longer than 1 letter
def has_latin(text):
    if sum(all_latin(w) for w in str(text).split()) > 2:
        return True
    else:
        return False

# Return words with Latin letters longer than 1 letter
def get_latin(text):
    latin_wds = [w for w in str(text).split() if all_latin(w)]
    if len(latin_wds) > 2:
        return latin_wds
    else:
        return None

In [3]:
cstalk_path = '/home/michael/school/research/wp/ar/cs_talk_scores.csv'
data = pd.read_csv(cstalk_path)
print(len(data))

cs_mask = data['editor_talk'].map(has_latin)
cs_rows = data[cs_mask]
print(len(cs_rows))
data

8607
2189


,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score
0,.امارات,تنبيه: وصلة ميتة تحقّق منها,جار الله,بعد عمليات آلية عدة، كُشِف أن الوصلة الخارجية ...,NaN,1,0,1.000000
1,18 فبراير,تنبيه,ASammour,":::TEMPLATE[ر, Taher2000] لو أنك تقصد الاستبدا...",هناك خطأ في التقويم، حيث أن أسماء الأيام لا تت...,2,11,1.000000
2,18 فبراير,تنبيه,Dr-Taher Khalid,"شكرا TEMPLATE[ر, YOUSSEF.AIT.CHAIB] للتنبيه. ا...",هناك خطأ في التقويم، حيث أن أسماء الأيام لا تت...,5,8,0.979592
3,18 فبراير,تنبيه,باسم,":::::TEMPLATE[أشار, Taher2000, ASammour] إن كا...",هناك خطأ في التقويم، حيث أن أسماء الأيام لا تت...,2,11,0.953488
4,1989 (ألبوم تايلور سويفت),تسمية المقال,حنان محمود,"TEMPLATE[ر, محمد الفلسطيني] TEMPLATE[تم, تم]. ...","يفترض أن تبقى التسمية على ما هي، أي ""1989 (ألب...",1,1,0.760187
5,21 شارع جامب,ترجمة أسم الفيلم,سامي الرحيلي,"الزملاء TEMPLATE[اشار, ابوهايدي, Shbib Al-Suba...",NaN,1,0,0.664502
6,300 (فيلم),العلاقة بين الفيلم وإيران والأحداث الجارية.,Aziz1,قمت باعادة صياغة جزء من فقرة الانتقاد .يمكنك أ...,أعتقد أنه يجب حذف أو تعديل الفقرة المتعلقة بإي...,3,3,1.000000
7,5 يناير,اقترح حذف الأحداث التالية,Kaothar,أقترح حذف الأحداث التالية حيث أنها لا تؤثر علي...,NaN,4,0,0.979381
8,آدم,إعادة كتابة,Aram33,الموضوع يحتاج لإعادة كتابة ليتوافق مع الصفة ال...,": انا معك اخي العزيز , و أحبذ وضع افكار الديان...",3,6,0.808511
9,آدم,إعادة كتابة,Wazup,الصفحة تقوم بتعريف ادم وفق الاديان السماويةعلى...,الموضوع يحتاج لإعادة كتابة ليتوافق مع الصفة ال...,3,6,0.176471


## See most common Latin-letter words

In [13]:
cs_wds = Counter([w for wds in cs_rows['editor_talk'].map(get_latin).tolist() for w in wds])
cs_wds.most_common(100)

[('the', 1606),
 ('of', 1071),
 ('and', 742),
 ('to', 715),
 ('on', 638),
 ('in', 627),
 ('In', 481),
 ('is', 454),
 ('Drork', 353),
 ('Not', 300),
 ('that', 300),
 ('not', 237),
 ('Found', 220),
 ('for', 220),
 ('as', 219),
 ('be', 209),
 ('The', 201),
 ('or', 196),
 ('by', 186),
 ('--abanima', 185),
 ('it', 182),
 ('de', 181),
 ('this', 172),
 ('Slacker', 159),
 ('are', 156),
 ('was', 154),
 ('with', 151),
 ('have', 138),
 ('Aziz', 135),
 ('Muhends', 134),
 ('from', 132),
 ('--Abdul', 132),
 ('you', 130),
 ('--HARRY', 127),
 ('--Dr-Taher', 115),
 ('article', 108),
 ('can', 106),
 ('an', 106),
 ('Yamanam', 94),
 ('Muhammad', 92),
 ('--Omaislam', 89),
 ('has', 84),
 ('at', 84),
 ('Egyptian', 80),
 ('--Skaterboy', 80),
 ('Arabic', 78),
 ('et', 78),
 ('other', 75),
 ('which', 74),
 ('their', 72),
 ('but', 71),
 ('Volume', 68),
 ("'Name", 68),
 ('service', 68),
 ('Socket', 68),
 ('Error:', 68),
 ('--A', 67),
 ('XVI', 66),
 ('Forbidden', 66),
 ('--Taher', 65),
 ('khayam', 65),
 ('].--JarBo

## Filter out conversations where everyone wins

In [4]:
threads = sorted(set(zip(data['article'], data['thread_title'])))
len(threads)

5451

In [5]:
lose_threads = []
threshold = 0.5

for art, t in threads:
    t_rows = data[(data['article']==art) & (data['thread_title']==t)]
    scores = t_rows['editor_score'].tolist()
    if not all([s > threshold for s in scores]):
        lose_threads.append((art,t))
        
print(len(lose_threads))

1974


In [6]:
mask = [tup in lose_threads for tup in zip(data['article'], data['thread_title'])]
lose_rows = data[mask]
len(lose_rows)

4223

In [7]:
lose_rows.to_csv('/home/michael/school/research/wp/ar/cs_talk_filtered.csv', index=False)

In [8]:
# See how many of filtered rows have cs
cs_mask = lose_rows['editor_talk'].map(has_latin)
print(sum(cs_mask))

1034


## Filter out singleton conversations

In [3]:
lose_rows = pd.read_csv('/home/michael/school/research/wp/ar/cs_talk_filtered.csv')
lose_rows

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score
0,آدم,إعادة كتابة,Aram33,الموضوع يحتاج لإعادة كتابة ليتوافق مع الصفة ال...,": انا معك اخي العزيز , و أحبذ وضع افكار الديان...",3,6,0.808511
1,آدم,إعادة كتابة,Wazup,الصفحة تقوم بتعريف ادم وفق الاديان السماويةعلى...,الموضوع يحتاج لإعادة كتابة ليتوافق مع الصفة ال...,3,6,0.176471
2,آريوسية,اريوس كان علي المسيحيه الصحيحه,Exact Team,أولاً كن مهذبًا ومؤدبًا، لست مضطرًا للرد على أ...,اريوس كان علي دين السيد المسيح وهوه لم يضيف او...,1,2,0.006070
3,آريوسية,اريوس كان علي المسيحيه الصحيحه,Sammy.aw,اريوس كان علي دين السيد المسيح وهوه لم يضيف او...,أولاً كن مهذبًا ومؤدبًا، لست مضطرًا للرد على أ...,2,1,0.803316
4,آريوسية,راجع,Exact Team,المواعظ والاعتبار للمقريزي\n” وأكرم قسطنطين ال...,:غير أن آريوس لم يخرج ببدعة جديدة في هذا التوج...,3,3,0.379363
5,آريوسية,راجع,هشام دياب,:غير أن آريوس لم يخرج ببدعة جديدة في هذا التوج...,المواعظ والاعتبار للمقريزي\n” وأكرم قسطنطين ال...,3,3,0.614613
6,آشوريون قدماء,أصل الاشوريين,Kathovo,الاشوريون لم يصلوا يوماً إلى اليمن. ربما كانت ...,المرجع ندوة في جامعة الملك محمد الخامس وهو إضا...,4,4,0.851648
7,آشوريون قدماء,أصل الاشوريين,Rami radwan,المرجع ندوة في جامعة الملك محمد الخامس وهو إضا...,الاشوريون لم يصلوا يوماً إلى اليمن. ربما كانت ...,4,4,0.110000
8,آشوريون قدماء,دمج مع مقالة آشور,Kathovo,اقترح دمج هذه المقالة بمقالة آشور حيث لا وجود ...,لا أحب خوض جدالات من هذا النوع ولكن لي تحفظات ...,6,6,0.851648
9,آشوريون قدماء,دمج مع مقالة آشور,Rami radwan,تحياتي ، لا مشكلة في الدمج بل أنا من مؤيدي الد...,اقترح دمج هذه المقالة بمقالة آشور حيث لا وجود ...,4,8,0.110000


In [4]:
lose_threads = list(set(zip(lose_rows['article'], lose_rows['thread_title'])))
len(lose_threads)

1974

In [6]:
singleton_counter = 0

for a, t in lose_threads:
    if len(lose_rows[(lose_rows['article'] == a) & (lose_rows['thread_title'] == t)]) == 1: 
        singleton_counter += 1
        
print(singleton_counter)

622


# See if cs talk pages segmented

In [2]:
# Load cs page names
cs_pagenames = pd.read_table('/home/michael/school/research/wp/ar/cs_pagenames.txt', header=None)[0].tolist()
len(cs_pagenames)

6384

In [9]:
# Load old parsed talk page data
all_talk = pd.read_csv('../ar/artalk_pages.csv') # this was all pages, not just cs pages
arts = set(all_talk['article_name'].tolist())
len(arts)

79177

In [4]:
# Load new parsed talk page data
all_talk = pd.read_csv('../ar/artalk_cspages_segmented.csv')
arts = set(all_talk['article_title'].tolist())
len(arts)

5623

In [6]:
# Load old parsed talk page data
old_cstalk = pd.read_csv('../ar/artalk_cs.csv')
old_arts = set(old_cstalk['article_title'].tolist())
len(old_arts)

1038

In [5]:
# Proportion of pages in cs pagenames that are in segmented cs pages
len(set(cs_pagenames) & arts) / (len(cs_pagenames))

0.6203007518796992

Not sure why this proportion is so low. Almost all the contributions in discoursedb_arwiki made it to the TSV file, so I assume it's in the JWPL extraction that it doesn't happen. But that importing (WikipediaTalkPageConverterApplication) is a bit obtuse, so don't know what's going on

In [7]:
# Proportion of pages in cs pagenames that are in old segmented cs pages
len(set(cs_pagenames) & old_arts) / (len(cs_pagenames))

0.075031328320802

In [22]:
len(all_talk)

45234

In [18]:
# Take out Discussion: prefix
all_talk['article_title'] = pd.Series([name[len('Discussion:'):] if name.startswith('Discussion:') else name for name in all_talk['article_title'].tolist()])
all_talk

,article_title,thread_title,timestamp,username,post_type,post_text
0,السلام عليك يا مريم,مصدر الصيغة الحالية,2007-02-08 23:55:40,Azdiyy,THREAD_STARTER,هل يمكن ذكر أقدم مصدر يذكرها بشكلها الحالي ولم...
1,قومية عربية,تصحيحاً لما عرف عن أسماعيل صلى الله عليه و سلم,2008-01-07 10:38:53,Mahaodeh,THREAD_STARTER,تصحيحاً لما عرف عن أسماعيل صلى الله عليه و سلم...
2,قومية عربية,لماذا لانضع الأسلام و الحضارة الأسلامية عنوانا...,2008-01-11 08:47:50,Lord Anubis,THREAD_STARTER,كما أشرت سابقاً أن أجدادنا ليس عرب و أنما عجم ...
3,قومية عربية,لماذا لانضع الأسلام و الحضارة الأسلامية عنوانا...,2008-01-07 10:38:53,Mahaodeh,POST,والعرب قلة من المسلمين. إن ثمانين بالمائه ممن ...
4,قومية عربية,نحن فعلاً أولاد عجم و يجب أحترام أصولنا و تاريخنا,2008-01-07 10:38:53,Mahaodeh,THREAD_STARTER,أنا مصرى من أصل أندلسى قوطى ولايجمعنى مع العرب...
5,قومية عربية,هذه الصفحة ليست منتدى,2008-01-07 10:38:53,Mahaodeh,THREAD_STARTER,أرجوا من الأخوان الذين يتمتعون بآراء مغايرة تج...
6,قومية عربية,إقتراح لتحسين المقال,2008-12-11 17:07:12,عبد الخالق رزق لله,THREAD_STARTER,أظن المقال يجب أن يحتوي على فقرة تبين الفرق بي...
7,قومية عربية,إقتراح لتحسين المقال,2008-01-07 10:38:53,Mahaodeh,POST,السلام عليكم ورحمة الله وبركاته\nأنا لاأتكلم ب...
8,قومية عربية,الخلط بين القومية العربية كحركة سياسية والعروب...,2008-01-07 10:38:53,Mahaodeh,THREAD_STARTER,يبدو أن كاتب المقال يقول أن أكثر المفكرين يقول...
9,قومية عربية,الخلط بين القومية العربية كحركة سياسية والعروب...,2009-02-04 13:54:41,Mahaodeh,POST,ثم أن المقال يتحدث عن القومية العربية فقط كحرك...


In [19]:
all_talk.to_csv('../ar/artalk_cspages_segmented.tsv', index=False)

# Find talk pages with CS

In [4]:
# See if a text has at least three words with Latin letters longer than 1 letter
def has_latin(text):
    if sum(all_latin(w) for w in str(text).split()) > 2:
        return True
    else:
        return False

In [3]:
# See if a word is all Latin characters
def all_latin(word):
    if len(word) > 1 and all(c in letters for c in word) and not word.startswith('http') \
            and not word.startswith('TEMPLATE'):
        return True
    else:
        return False

In [36]:
# Load talk pages
all_talk = pd.read_csv('../ar/artalk_pages.csv', error_bad_lines=True)

print(len(all_talk))
all_talk

79566


,article_name,discussion_text
0,ماء,اقترح صورة اكثر تعبيراً. الصورة المرفقة فنية ا...
1,رياضيات,\n\nمن هو جابر بن حيان؟ أنظر http://66.102.7.1...
2,الصفحة الرئيسية,الأخبار المرجو إضافة خبر وفاة رئيس الفيفا السا...
3,إستونيا,"Greetings. Several interwikis articles on ""geo..."
4,تلفاز,تلفاز وليس الرائي هي التسمية التي توصل إليها ا...
5,قرطاج,مسودة وفي نهاية القرن التاسع عشر، انتصبت الحما...
6,الأحداث الجارية,لاتلعنوا ظلام ويكيبيديا بل أضيئوا شمعة أولا...
7,شعر (أدب),تعديلات البند الرابع من المقدمة يحتوي على بعض ...
8,عمان (مدينة),\nمركز|150بكالأرشيف: «۱» شعار امانة عمان الجدي...
9,ابن خلدون,١ييج‌ڃٳd͡z„“ تقييم صفحة ابن خلدون لماذا هذه ال...


In [44]:
cs_talk = all_talk[all_talk['discussion_text'].map(has_latin)]
cs_pagenames = sorted(set([p[len('Discussion:'):] if p.startswith('Discussion:') else p for p in cs_talk['article_name'].tolist()]))
len(cs_pagenames)

6384

In [46]:
# Print names of pages with cs
cs_pagenames_outpath = '/home/michael/school/research/wp/ar/cs_pagenames.txt'
with open(cs_pagenames_outpath, 'w') as f:
    for name in cs_pagenames:
        f.write(name + '\n')

# Try feature selection

In [15]:
# Load features
feats = pd.read_csv('../ar/cs_discussion_features+10000.csv')
print(len(feats))

162


In [16]:
# Select columns
# ed_nonbow_cols = ['#editor_turns', 'en_cs', 'editor_prop_en', 'editor_prop_switches', 'editor_two_quotes', 
#                   'editor_en_technical']
# other_nonbow_cols = ['#other_turns', 'other_en_cs', 'other_prop_en', 'other_prop_switches', 'other_two_quotes', 
#                   'other_en_technical']
ed_nonbow_cols = ['en_cs', 'editor_prop_en', 'editor_prop_switches', 'editor_two_quotes', 
                  'editor_en_technical']
other_nonbow_cols = ['other_en_cs', 'other_prop_en', 'other_prop_switches', 'other_two_quotes', 
                  'other_en_technical']

In [24]:
# Vectorize input features
feats_v = {}

# Get unigram features
# v = CountVectorizer(min_df=1, stop_words='english')
v = TfidfVectorizer(min_df=1, stop_words='english')
edbow = v.fit_transform(feats['editor_talk'])
print(edbow.shape)

# v_other = CountVectorizer(min_df=1, stop_words='english')
v_other = TfidfVectorizer(min_df=1, stop_words='english')
other_bow = v_other.fit_transform(feats['other_talk'])
print(other_bow.shape)

# Get exclusive editor non-unigram features
ed_nonbow_d = {}
for col in ed_nonbow_cols:
    ed_nonbow_d[col] = np.array([feats[col]]).T
#     ed_nonbow_d[col] = np.array([(v - min(feats[col]))/max(feats[col]) for v in feats[col].values]).T
ed_nonbow = np.hstack(ed_nonbow_d.values())
print(ed_nonbow.shape)

# Get others' non-unigram features
nonbow_d = {}
for col in other_nonbow_cols:
    nonbow_d[col] = np.array([feats[col]]).T
#     nonbow_d[col] = np.array([(v - min(feats[col]))/max(feats[col]) for v in feats[col].values]).T
nonbow = np.hstack(nonbow_d.values())
print(nonbow.shape)

# Assemble editor features
#edfeats = np.hstack([edbow, ed_nonbow])
edfeats = hstack([edbow, ed_nonbow])

# Assemble non-unigram features
nonbow_f = np.hstack([ed_nonbow, nonbow])

# Assemble unigram features
bow_f = hstack([edbow, other_bow])
print(bow_f.shape)

# Assemble all features
feats_v = hstack([edbow, other_bow, ed_nonbow, nonbow])
feats_v.shape

(162, 7732)
(162, 9092)
(162, 5)
(162, 5)
(162, 16824)


(162, 16834)

In [25]:
train_types = [edbow, ed_nonbow, edfeats, bow_f, nonbow_f, feats_v]
for train_data in train_types:
    clf = make_pipeline(SelectPercentile(chi2, percentile=20), LogisticRegression(class_weight='balanced'))
    loo = LeaveOneOut().split(train_data)
    scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
    print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.549
Accuracy: 0.556
Accuracy: 0.599
Accuracy: 0.568
Accuracy: 0.543
Accuracy: 0.630


In [27]:
train_types = [edbow, ed_nonbow, edfeats, bow_f, nonbow_f, feats_v]
for train_data in train_types:
    clf = make_pipeline(SelectPercentile(chi2, percentile=100), LogisticRegression(class_weight='balanced'))
    loo = LeaveOneOut().split(train_data)
    scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
    print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.586
Accuracy: 0.580
Accuracy: 0.617
Accuracy: 0.556
Accuracy: 0.302
Accuracy: 0.574


# Merge random talk page sample and cs

In [59]:
# Load features
cs_feats = pd.read_csv('../ar/cs_discussion_features.csv')
print(len(cs_feats))
other_feats = pd.read_csv('../ar/discussion_features_10000.csv')
print(len(other_feats))

85
147


In [60]:
merged = pd.concat([cs_feats, other_feats])
merged.drop_duplicates(subset=['article','thread_title','editor'],inplace=True)
merged = merged[merged['#other_turns']>0] # Get rid of any solo editors
print(len(merged))

merged = merged[list(cs_feats.columns)]
merged

162


,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical,editor_score,editor_success
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000,0.932735,True
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,False,True,0.000000,0.341682,0.000000,0.209346,False,True,0.000000,0.065934,1.000000,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000,0.932735,True
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,False,True,0.000000,0.181818,0.000000,0.113514,False,True,0.000000,0.088235,1.000000,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,False,True,0.000000,0.126866,0.000000,0.080153,False,True,0.000000,0.088235,0.205736,False
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,False,True,0.090909,0.008913,0.200000,0.010870,False,True,1.000000,0.166667,0.930295,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,False,True,0.000000,0.011321,0.000000,0.015296,False,True,0.000000,0.285714,1.000000,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,False,True,0.000000,0.012270,0.000000,0.016632,False,True,0.000000,0.285714,0.236348,False
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,False,True,0.000000,0.019868,0.000000,0.013699,False,True,0.000000,0.000000,0.231527,False
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,True,False,0.025000,0.000000,0.016949,0.000000,True,False,0.000000,0.000000,0.801391,True


In [61]:
merged.to_csv('../ar/cs_discussion_features+10000.csv', index=False)

In [68]:
any(pd.isnull(merged['other_talk']))

False

# Randomly sample talk page

In [3]:
talk = pd.read_csv('../ar/artalk.tsv', delimiter='\t')
len(talk)

22870

In [52]:
n_rows = 10000
talk_sample = talk.sample(n=n_rows)
print(len(talk_sample))

talk_sample.to_csv('../ar/artalk_{:d}.csv'.format(n_rows), index=False)

10000


# Load cs features

In [4]:
# Load features
feats = pd.read_csv('../ar/cs_discussion_features.csv')
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical,editor_score,editor_success
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000,0.932735,True
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,False,True,0.000000,0.341682,0.000000,0.209346,False,True,0.000000,0.065934,1.000000,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000,0.932735,True
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,False,True,0.000000,0.181818,0.000000,0.113514,False,True,0.000000,0.088235,1.000000,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,False,True,0.000000,0.126866,0.000000,0.080153,False,True,0.000000,0.088235,0.205736,False
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,False,True,0.090909,0.008913,0.200000,0.010870,False,True,1.000000,0.166667,0.930295,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,False,True,0.000000,0.011321,0.000000,0.015296,False,True,0.000000,0.285714,1.000000,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,False,True,0.000000,0.012270,0.000000,0.016632,False,True,0.000000,0.285714,0.236348,False
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,False,True,0.000000,0.019868,0.000000,0.013699,False,True,0.000000,0.000000,0.231527,False
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,True,False,0.025000,0.000000,0.016949,0.000000,True,False,0.000000,0.000000,0.801391,True


# Test technical En features

In [5]:
tech = feats[feats['editor_en_technical'] > 0]
print(len(tech))
tech

14


,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical,editor_score,editor_success
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000,0.932735,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000,0.932735,True
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,False,True,0.090909,0.008913,0.200000,0.010870,False,True,1.000000,0.166667,0.930295,True
10,أمازيغ,الهند وباكستان R1 و R2,Dukkani,سلالات الهند وباكستان تتوزع بشكل كبير بين السل...,الأبحاث العلمية خرقت هذه القاعدة في أكثر مناطق...,4,3,True,False,0.102410,0.000000,0.167742,0.000000,False,False,0.200000,0.000000,0.284389,False
13,إندورفين,سوء فهم,Mohdalg,هذا غير صحيح ... قرأت المقالتان في الموقعين ال...,ما أراه في ويكيبيديا الإنكليزية حالياً لا يؤكد...,2,3,True,False,0.235772,0.000000,0.147186,0.000000,False,False,0.050847,0.000000,0.953036,True
24,القمر,مقدمة تعريفية أساسية مفقودة في مقالة مختارة؟,عباد ديرانية,لقد لاحظت أن مقدمة هذه المقالة المختارة لا تحت...,احدى اهم الحقائق التي تنقض هذه النظرية هي اختل...,6,8,True,False,0.016129,0.000000,0.009412,0.000000,True,False,0.125000,0.000000,0.636843,True
26,اللات,صحة المعلومات,Yuba~arwiki,"بهذا الشكل لن تفيد أحدا, اطرح وجهة نظرك, وقدم ...",:بالنسبة للات، فما وجه الربط بين النجمة (رمز ا...,10,21,True,True,0.091379,0.053992,0.091743,0.026377,True,True,0.040000,0.074627,0.456332,False
34,سدر,السدر في المصادر,Makki98,معظم المصادر تشير إلى أن السدر هو النوع Ziziph...,لذلك، مقالة سدر تكون مقابل النوع زفيزف شوك الم...,2,2,True,False,0.035587,0.000000,0.061069,0.000000,False,False,0.636364,0.000000,0.735043,True
39,سرطان,المصطلحات الانكليزية في المقالة,MD Syria,أنا أفضل إزالة المصطلحات الإنجليزية فمثلا العل...,أرجوا من الزملاء إعطاء رأي بالموضوع--A e t h e...,3,12,True,False,0.018750,0.000000,0.037736,0.000000,False,False,0.500000,0.000000,0.766187,True
43,سيبرنيطيقا,الى مبتدئ,مبتدئ,:: شكرا خالد على الإقتراحات تمنيت أن يكون الفر...,"هل ممكن وضع الجزء الخاص بفينر في مقالته:""ولد ف...",1,2,True,False,0.047945,0.000000,0.045455,0.000000,False,False,0.416667,0.000000,0.757576,True


In [6]:
pd.options.display.max_colwidth = 0

In [8]:
tech.loc[:,['editor_talk', 'en_cs', 'editor_two_quotes', 'editor_en_technical']].sort_values(['editor_en_technical'], ascending=False)

editor_talk  \
5   Y-chromosomal Aaron\nY-chromosomal Adam\nMitochondrial Eve --Dukkani (نقاش) 12:55، 27 يناير 2009 (UTC)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# Example instances

## CS and success

In [3]:
cs_posts = feats[feats['en_cs']==True]
noncs_posts = feats[feats['en_cs']==False]
sorted_cs_posts = cs_posts.sort_values(['editor_score'], ascending=False)
sorted_cs_posts

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical,editor_score,editor_success
13,إندورفين,سوء فهم,Mohdalg,هذا غير صحيح ... قرأت المقالتان في الموقعين ال...,ما أراه في ويكيبيديا الإنكليزية حالياً لا يؤكد...,2,3,True,False,0.235772,0.000000,0.147186,0.000000,False,False,0.050847,0.000000,0.953036,True
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000,0.932735,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000,0.932735,True
44,فلسفة,تعليق أحد المشاركين,Dr-Taher Khalid,التعليق التالي كتبه مستخدم: محمد نور بروق في ص...,http://im46.gulfup.com/Bnc1Xm.jpg\nتعليق1\nتعليق2,1,1,True,False,0.065574,0.000000,0.101695,0.000000,False,False,0.000000,0.000000,0.923387,True
15,إندونيسيا,المصادر,Mahmoudalrawi,w:Demographics of Indonesia = سكان إندونيسيا w...,1. المصادر غير معروفة لأنها مأخوذة من ويكيبيدي...,1,1,True,False,0.111111,0.000000,0.285714,0.000000,False,False,0.000000,0.000000,0.806718,True
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,True,False,0.025000,0.000000,0.016949,0.000000,True,False,0.000000,0.000000,0.801391,True
39,سرطان,المصطلحات الانكليزية في المقالة,MD Syria,أنا أفضل إزالة المصطلحات الإنجليزية فمثلا العل...,أرجوا من الزملاء إعطاء رأي بالموضوع--A e t h e...,3,12,True,False,0.018750,0.000000,0.037736,0.000000,False,False,0.500000,0.000000,0.766187,True
43,سيبرنيطيقا,الى مبتدئ,مبتدئ,:: شكرا خالد على الإقتراحات تمنيت أن يكون الفر...,"هل ممكن وضع الجزء الخاص بفينر في مقالته:""ولد ف...",1,2,True,False,0.047945,0.000000,0.045455,0.000000,False,False,0.416667,0.000000,0.757576,True
34,سدر,السدر في المصادر,Makki98,معظم المصادر تشير إلى أن السدر هو النوع Ziziph...,لذلك، مقالة سدر تكون مقابل النوع زفيزف شوك الم...,2,2,True,False,0.035587,0.000000,0.061069,0.000000,False,False,0.636364,0.000000,0.735043,True
29,دمشق,المراجع,Sammy.aw,هو في الواقع، أنا كنت أستخدم هذا النوع من الأر...,أظنها فكرة حسنة. مع التحيات. عمرو بن كلثوم (لل...,1,3,True,False,0.095890,0.000000,0.028571,0.000000,False,False,0.000000,0.000000,0.679814,True


In [5]:
# Save sorted cs posts
sorted_cs_posts.to_csv('../ar/cs_posts.csv', index=False)

In [10]:
for i in range(10):
    print(i, end=': ')
    print(sorted_cs_posts.iloc[i]['editor_talk'])
    print()

0: هذا غير صحيح ... قرأت المقالتان في الموقعين المذكورين و لم أجد ما يقول أن هرمون الإندورفين هو الهرمون المنشط للغدة الكظرية!! مصدرهما واحد لكنهما يختلفون في التركيبة الكيميائية و في وظائفهما. Mohdalg (نقاش) 22:21، 27 فبراير 2009 (ت‌ع‌م) أخي منار... الكيمياء الحيوية و اعتلال الهرمونات كانت جزء كبير من دراستي الجامعية و قد قرأت الكثير من الكتب في هذا الموضوع. و قبل أن أعدل المقال أو أكتب في النقاش تأكدت من معلوماتي من أكثر من مصدر. و كلها ترد على نفس النتيجة. و لم أقرأ في مكان ما كتبته أنت إلا في موقعك الإلكتروني! و نقلاً عن ويكيبيديا الإنكليزية لم يتم ذكر هرمون الACTH إلا في موضعين و أنقلها حرفياً: The physiological importance of the beta-endorphin that can be measured in the blood is far from clear: beta-endorphin is a cleavage product of pro-opiomelanocortin (POMC) which is also the precursor hormone for adrenocorticotrophic hormone (ACTH). The behavioural effects of beta-endorphin are exerted by its actions in the brain and spinal cord, and probably the hypothalamic neurons are the

## CS and failure

In [12]:
for i in range(1,6):
    print(-1* i, end=': ')
    print(sorted_cs_posts.iloc[-1*i]['editor_talk'])
    print()

-1: السلام عليكم، من المرجو شرح الإلغاءات الغير المبررة فيما يتعلق اضافة مركز بحث في المقالة؟!--موريسكو (نقاش) 16:38، 29 ديسمبر 2013 (ت ع م) ::ليكن في علمكم أنّ  المركز  لديه ماجستير و مدرسة الدكتوراه (école doctorale). تحياتي--موريسكو (نقاش) 22:38، 29 ديسمبر 2013 (ت ع م) ::::Salam, Les missions principales du centre sont:\nla recherche ;\n    la formation ;\n    les études et expertises ;\n    l'édition et la valorisation.

-2: سلالات الهند وباكستان تتوزع بشكل كبير بين السلالات والسلالة الأكثر حضورا بين السلالات تحضر عند كلا الشعبين وهي R1a1 ثم R2 مما يوضح أنها سلالات هندية\nY-Chromosome Distributions in India\n--Dukkani (نقاش) 17:19، 16 أغسطس 2009 (ت‌ع‌م) معاليكم لا يريد أن يسمع أن أهم سلالة في إيران هي J :\nIn northern Iran, group J represents the most frequent\nhaplogroup (33.33%) followed by groups R (27.27%), G\n(15.15%), Q (9.09%) and N (6.06%). Similarly, in southern\nIran group J exhibits the highest frequency (35.04%) and\nis trailed by groups R (25.64%), G (12.82%), E (6.84%

# T test for diff bw means

In [18]:
# Split based on en_cs
cs_posts = feats[feats['en_cs']==True]
noncs_posts = feats[feats['en_cs']==False]
print(len(cs_posts))
print(len(noncs_posts))

ttest_ind(cs_posts['editor_score'], noncs_posts['editor_score'])

print(np.mean(cs_posts['editor_score']))
print(np.mean(noncs_posts['editor_score']))

17
68
0.680579304467
0.510664691015


## Variation among cs features

In [6]:
# Discrete features--t test
for f in feats.columns[8:10] | feats.columns[-4:-2]:
    print(f)
    fpresent = feats[feats[f]==True]
    nof = feats[feats[f]==False]
    print("Number of positive feature example: {:d}".format(len(fpresent)))
    print(ttest_ind(fpresent['editor_score'], nof['editor_score']))
    print()

editor_two_quotes
Number of positive feature example: 7
Ttest_indResult(statistic=1.4590894143707476, pvalue=0.14831477033907189)

en_cs
Number of positive feature example: 17
Ttest_indResult(statistic=2.0306424329671411, pvalue=0.045492751687925027)

other_en_cs
Number of positive feature example: 39
Ttest_indResult(statistic=-0.53229325899209945, pvalue=0.59594498343111113)

other_two_quotes
Number of positive feature example: 27
Ttest_indResult(statistic=-0.772365080790132, pvalue=0.44209199899605134)



In [9]:
# Continuous features (measure correlation)
for f in feats.columns[10:14] | feats.columns[-2:]:
    print(f)
    print(np.nonzero(feats[f])[0].shape)
    print(pearsonr(feats[f], feats['editor_score']))
    print()

editor_en_technical
(14,)
(0.17137233662776372, 0.11683289555735298)

editor_prop_en
(19,)
(0.19204111306801791, 0.078281618340537568)

editor_prop_switches
(19,)
(0.20692629470403437, 0.057414784691648346)

other_en_technical
(32,)
(-0.058270386514153082, 0.59630435285818906)

other_prop_en
(42,)
(-0.083365413716951567, 0.44813205380456844)

other_prop_switches
(42,)
(-0.018959535012834455, 0.863260169249918)



In [37]:
feats.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'en_cs', 'other_en_cs',
       'editor_prop_en', 'other_prop_en', 'editor_prop_switches',
       'other_prop_switches', 'editor_two_quotes', 'other_two_quotes',
       'editor_en_technical', 'other_en_technical', 'editor_score',
       'editor_success'],
      dtype='object')

In [38]:
# Select columns
ed_nonbow_cols = ['#editor_turns', 'en_cs', 'editor_prop_en', 'editor_prop_switches', 'editor_two_quotes', 
                  'editor_en_technical']
other_nonbow_cols = ['#other_turns', 'other_en_cs', 'other_prop_en', 'other_prop_switches', 'other_two_quotes', 
                  'other_en_technical']

In [39]:
# Vectorize input features
feats_v = {}

# Get unigram features
v = CountVectorizer(min_df=1, stop_words='english')
edbow = v.fit_transform(feats['editor_talk'])
print(edbow.shape)

v_other = CountVectorizer(min_df=1, stop_words='english')
other_bow = v_other.fit_transform(feats['other_talk'])
print(other_bow.shape)

# Get exclusive editor non-unigram features
ed_nonbow_d = {}
for col in ed_nonbow_cols:
    ed_nonbow_d[col] = np.array([feats[col]]).T
#     ed_nonbow_d[col] = np.array([(v - min(feats[col]))/max(feats[col]) for v in feats[col].values]).T
ed_nonbow = np.hstack(ed_nonbow_d.values())
print(ed_nonbow.shape)

# Get others' non-unigram features
nonbow_d = {}
for col in other_nonbow_cols:
    nonbow_d[col] = np.array([feats[col]]).T
#     nonbow_d[col] = np.array([(v - min(feats[col]))/max(feats[col]) for v in feats[col].values]).T
nonbow = np.hstack(nonbow_d.values())
print(nonbow.shape)

# Assemble editor features
#edfeats = np.hstack([edbow, ed_nonbow])
edfeats = hstack([edbow, ed_nonbow])

# Assemble non-unigram features
nonbow_f = np.hstack([ed_nonbow, nonbow])

# Assemble unigram features
bow_f = hstack([edbow, other_bow])
print(bow_f.shape)

# Assemble all features
feats_v = hstack([edbow, other_bow, ed_nonbow, nonbow])
feats_v.shape

(85, 5383)
(85, 6106)
(85, 6)
(85, 6)
(85, 11489)


(85, 11501)

# Linear regression

## Train and test editor features

In [63]:
# Train and test linear regression classifier
clf = LinearRegression()

# edpred = cross_validation.cross_val_predict(clf, edfeats, feats['editor_score'], cv=10)
edpred = cross_validation.cross_val_predict(clf, ed_nonbow, feats['editor_score'], cv=10)
edpred = np.array([max(p, 0.0) for p in edpred])
edpred = np.array([min(p, 1.0) for p in edpred])

# The mean square error
print("mse: {:2f}".format(np.mean((edpred - feats['editor_score'].values) ** 2)))

# The root mean square error
print("rmse: {:2f}".format(math.sqrt(np.mean((edpred - feats['editor_score'].values) ** 2))))

mse: 0.125238
rmse: 0.353890


## Try 0.5 baseline

In [5]:
# The mean square error
guess = [0.5]*len(feats['editor_score'])
print("mse: {:2f}".format(np.mean((guess - feats['editor_score'].values) ** 2)))

# The root mean square error
print("rmse: {:2f}".format(math.sqrt(np.mean((guess - feats['editor_score'].values) ** 2))))

mse: 0.099593
rmse: 0.315584


## Train and test editor+other features (all features)

In [16]:
# Train and test linear regression classifier on all features (continuous editor_score input)
clf = LinearRegression()

pred = cross_validation.cross_val_predict(clf, feats_v, feats['editor_score'], cv=10)

bounded_pred = np.array([max(p, 0.0) for p in pred])
bounded_pred = np.array([min(p, 1.0) for p in bounded_pred])

# The mean square error
print("mse: {:2f}".format(np.mean((bounded_pred - feats['editor_score'].values) ** 2)))
print("rmse: {:2f}".format(math.sqrt(np.mean((bounded_pred - feats['editor_score'].values) ** 2))))

mse: 0.149273
rmse: 0.386359


# Logistic regression

## Leave-one-out CV

In [49]:
# Train and test logistic regression classifier on all features (continuous editor_score input)
train_data = edbow
clf = LogisticRegression()
loo = LeaveOneOut().split(train_data)
scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.447


In [59]:
# Train and test logistic regression classifier on all features (continuous editor_score input)
train_data = ed_nonbow
clf = LogisticRegression()
loo = LeaveOneOut().split(train_data)
scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.624


In [58]:
# Train and test logistic regression classifier on all features (continuous editor_score input)
clf = LogisticRegression()
loo = LeaveOneOut().split(edfeats)
scores = cross_validation.cross_val_score(clf, edfeats, feats['editor_success'], cv=loo)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.459


In [60]:
# Train and test logistic regression classifier on all features (continuous editor_score input)
train_data = bow_f
clf = LogisticRegression()
loo = LeaveOneOut().split(train_data)
scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.435


In [61]:
# Train and test logistic regression classifier on all features (continuous editor_score input)
train_data = nonbow_f
clf = LogisticRegression()
loo = LeaveOneOut().split(train_data)
scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.494


In [62]:
# Train and test logistic regression classifier on all features (continuous editor_score input)
train_data = feats_v
clf = LogisticRegression()
loo = LeaveOneOut().split(train_data)
scores = cross_validation.cross_val_score(clf, train_data, feats['editor_success'], cv=loo)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.471


## 10-fold CV

In [53]:
# Train and test logistic regression classifier
clf = LogisticRegression()
scores = cross_validation.cross_val_score(clf, edbow, feats['editor_success'], cv=10)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.387


In [54]:
# Train and test logistic regression classifier
clf = LogisticRegression()
scores = cross_validation.cross_val_score(clf, edfeats, feats['editor_success'], cv=10)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.374


In [55]:
# Train and test logistic regression classifier
clf = LogisticRegression()
scores = cross_validation.cross_val_score(clf, bow_f, feats['editor_success'], cv=10)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.316


In [56]:
# Train and test logistic regression classifier
clf = LogisticRegression()
scores = cross_validation.cross_val_score(clf, feats_v, feats['editor_success'], cv=10)
print("Accuracy: %0.3f" % (scores.mean()))

Accuracy: 0.304


# Extract code-switching features

In [3]:
featfile = '/home/michael/school/research/wp/ar/cs_discussion_features.csv'
feats = pd.read_csv(featfile)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical,editor_score,editor_success
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000,0.932735,True
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,False,True,0.000000,0.341682,0.000000,0.209346,False,True,0.000000,0.065934,1.000000,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000,0.932735,True
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,False,True,0.000000,0.181818,0.000000,0.113514,False,True,0.000000,0.088235,1.000000,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,False,True,0.000000,0.126866,0.000000,0.080153,False,True,0.000000,0.088235,0.205736,False
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,False,True,0.090909,0.008913,0.200000,0.010870,False,True,1.000000,0.166667,0.930295,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,False,True,0.000000,0.011321,0.000000,0.015296,False,True,0.000000,0.285714,1.000000,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,False,True,0.000000,0.012270,0.000000,0.016632,False,True,0.000000,0.285714,0.236348,False
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,False,True,0.000000,0.019868,0.000000,0.013699,False,True,0.000000,0.000000,0.231527,False
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,True,False,0.025000,0.000000,0.016949,0.000000,True,False,0.000000,0.000000,0.801391,True


In [20]:
feats.drop(['editor_en_apologize', 'other_en_apologize'], axis=1, inplace=True)

In [21]:
feats.to_csv(featfile, index=False)

## technical terms in En
Doesn't have to have code-switching

In [6]:
def en_technical(text):
#     if not has_english(text): return 0.0
    en_wds = [w for w in str(text).split() if len(w) > 1 and w.lower() in endict and w != 'TEMPLATE']
    if len(en_wds) == 0:
        return 0.0
    caps = sum(1 for w in en_wds if w[0].isupper())
    return caps/len(en_wds)

In [9]:
ed_crit = [0.0 if not a and b else a for a,b in zip(feats['editor_talk'].map(en_technical), feats['en_cs'].tolist())]
other_crit = [0.0 if not a and b else a for a,b in zip(feats['other_talk'].map(en_technical), feats['other_en_cs'].tolist())]

feats['editor_en_technical'] = ed_crit
feats['other_en_technical'] = other_crit
feats.to_csv(featfile, index=False)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True,0.000000,0.341682,0.000000,0.209346,False,True,0.000000,0.065934
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True,0.000000,0.181818,0.000000,0.113514,False,True,0.000000,0.088235
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True,0.000000,0.126866,0.000000,0.080153,False,True,0.000000,0.088235
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True,0.090909,0.008913,0.200000,0.010870,False,True,1.000000,0.166667
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True,0.000000,0.011321,0.000000,0.015296,False,True,0.000000,0.285714
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True,0.000000,0.012270,0.000000,0.016632,False,True,0.000000,0.285714
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True,0.000000,0.019868,0.000000,0.013699,False,True,0.000000,0.000000
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False,0.025000,0.000000,0.016949,0.000000,True,False,0.000000,0.000000


In [47]:
sum(feats['editor_en_technical'].tolist())

11.0

## apologizing for using En--only 2 thread titles

In [11]:
def en_apologize(text):
    return ('English' in str(text) or 'english' in str(text)) and ('sorry' in str(text) or 'Sorry' in str(text))

In [12]:
ed_crit = [a and b for a,b in zip(feats['editor_talk'].map(en_apologize), feats['en_cs'].tolist())]
other_crit = [a and b for a,b in zip(feats['other_talk'].map(en_apologize), feats['other_en_cs'].tolist())]

feats['editor_en_apologize'] = ed_crit
feats['other_en_apologize'] = other_crit
feats.to_csv(featfile, index=False)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_apologize,other_en_apologize
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False,0.341682,0.000000,0.209346,0.000000,True,False,False,False
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True,0.000000,0.341682,0.000000,0.209346,False,True,False,False
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False,0.253731,0.000000,0.159091,0.000000,True,False,False,False
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True,0.000000,0.181818,0.000000,0.113514,False,True,False,False
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True,0.000000,0.126866,0.000000,0.080153,False,True,False,False
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True,0.090909,0.008913,0.200000,0.010870,False,True,False,False
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True,0.000000,0.011321,0.000000,0.015296,False,True,False,False
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True,0.000000,0.012270,0.000000,0.016632,False,True,False,False
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True,0.000000,0.019868,0.000000,0.013699,False,True,False,False
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False,0.025000,0.000000,0.016949,0.000000,True,False,False,False


In [14]:
sum(other_crit)

0

## quoting
Must also have code-switching

In [6]:
def two_quotes(text):
    return str(text).count('"') >= 2

In [8]:
ed_crit = [a and b for a,b in zip(feats['editor_talk'].map(two_quotes), feats['en_cs'].tolist())]
other_crit = [a and b for a,b in zip(feats['other_talk'].map(two_quotes), feats['other_en_cs'].tolist())]

feats['editor_two_quotes'] = ed_crit
feats['other_two_quotes'] = other_crit
feats.to_csv(featfile, index=False)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False,0.341682,0.000000,0.209346,0.000000,True,False
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True,0.000000,0.341682,0.000000,0.209346,False,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False,0.253731,0.000000,0.159091,0.000000,True,False
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True,0.000000,0.181818,0.000000,0.113514,False,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True,0.000000,0.126866,0.000000,0.080153,False,True
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True,0.090909,0.008913,0.200000,0.010870,False,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True,0.000000,0.011321,0.000000,0.015296,False,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True,0.000000,0.012270,0.000000,0.016632,False,True
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True,0.000000,0.019868,0.000000,0.013699,False,True
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False,0.025000,0.000000,0.016949,0.000000,True,False


## proportion of switches
Doesn't have to have code-switching

In [39]:
def prop_switches(text):
    en = [w in endict for w in str(text).split() if len(w) > 1]
    n_switches = sum(en[i] != en[i+1] for i in range(len(en)-1))
    if len(en) > 1:
        return n_switches/(len(en)-1)
    else:
        return 0.0

In [40]:
ed_crit = feats['editor_talk'].map(prop_switches)
other_crit = feats['other_talk'].map(prop_switches)
feats['editor_prop_switches'] = ed_crit
feats['other_prop_switches'] = other_crit
feats.to_csv(featfile, index=False)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False,0.341682,0.000000,0.209346,0.000000,True,False
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True,0.000000,0.341682,0.000000,0.209346,False,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False,0.253731,0.000000,0.159091,0.000000,True,False
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True,0.000000,0.181818,0.000000,0.113514,False,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True,0.000000,0.126866,0.000000,0.080153,False,True
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True,0.090909,0.008913,0.200000,0.010870,False,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True,0.000000,0.011321,0.000000,0.015296,False,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True,0.000000,0.012270,0.000000,0.016632,False,True
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True,0.000000,0.019868,0.000000,0.013699,False,True
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False,0.025000,0.000000,0.016949,0.000000,True,False


## proportion of English words

In [36]:
# See if a text has at least three English words
def prop_english(text):
    n_en = sum(w in endict for w in str(text).split() if len(w) > 1)
    return n_en/len(str(text).split())

In [37]:
crit = feats['editor_talk'].map(prop_english)
feats['editor_prop_en'] = crit
feats.to_csv(featfile, index=False)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False,0.341682,0.000000,0.220561,0.000000,True,False
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True,0.000000,0.325581,0.000000,0.220561,False,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False,0.253731,0.000000,0.219697,0.000000,True,False
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True,0.000000,0.181818,0.000000,0.156757,False,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True,0.000000,0.126866,0.000000,0.110687,False,True
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True,0.090909,0.010695,0.200000,0.009058,False,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True,0.000000,0.013208,0.000000,0.015296,False,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True,0.000000,0.014315,0.000000,0.016632,False,True
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True,0.000000,0.013245,0.000000,0.027397,False,True
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False,0.025000,0.000000,0.033898,0.000000,True,False


In [38]:
crit = feats['other_talk'].map(prop_english)
feats['other_prop_en'] = crit
feats.to_csv(featfile, index=False)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False,0.341682,0.000000,0.220561,0.000000,True,False
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True,0.000000,0.341682,0.000000,0.220561,False,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False,0.253731,0.000000,0.219697,0.000000,True,False
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True,0.000000,0.181818,0.000000,0.156757,False,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True,0.000000,0.126866,0.000000,0.110687,False,True
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True,0.090909,0.008913,0.200000,0.009058,False,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True,0.000000,0.011321,0.000000,0.015296,False,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True,0.000000,0.012270,0.000000,0.016632,False,True
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True,0.000000,0.019868,0.000000,0.027397,False,True
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False,0.025000,0.000000,0.033898,0.000000,True,False


## other_en_cs

In [3]:
# See if a text has at least three English words (don't have to be unique)
def has_english(text):
    if sum(w in endict for w in str(text).split() if len(w) > 1) > 2:
        return True
    else:
        return False

In [4]:
# Posts with code-switching
crit = feats['other_talk'].map(has_english)
feats['other_en_cs'] = crit
print("Number of other posts with English: {0} / {1}".format(sum(crit), len(feats)))
feats

Number of other posts with English: 39 / 85


,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs,other_en_cs
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True,False
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True,False
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False,True
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False,True
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False,True
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True,False


In [5]:
feats.to_csv(featfile, index=False)

## en_cs

In [3]:
# See if a text has at least three English words
def has_english(text):
    if sum(w in endict for w in str(text).split() if len(w) > 1) > 2:
        return True
    else:
        return False

In [4]:
# Posts with code-switching
crit = feats['editor_talk'].map(has_english)
feats['en_cs'] = crit
print("Number of posts with English: {0} / {1}".format(sum(crit), len(feats)))

Number of posts with English: 17 / 85


In [5]:
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,en_cs
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735,True
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000,False
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735,True
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000,False
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736,False
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,0.930295,False
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,1.000000,False
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,0.236348,False
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,0.231527,False
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,0.801391,True


In [6]:
feats.to_csv(featfile, index=False)

# See how many talk pages have missed

In [ ]:
# Load list of cs talk pages -- artalk_cs.csv is one that created, artalk_pages is one with full pages
talk = pd.read_csv('/home/michael/school/research/wp/ar/artalk.csv', parse_dates=['timestamp'])
cs_cur_rev = pd.read_csv('/home/michael/school/')

# Stats on cs discussion features

In [28]:
# featfile = '/home/michael/school/research/wp/ar/cs_discussion_features.csv'
# featfile = '/home/michael/school/research/wp/ar/cs_discussion_features+2000.csv'
featfile = '/home/michael/school/research/wp/ar/cs_discussion_features+10000.csv'
feats = pd.read_csv(featfile)
feats

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,en_cs,other_en_cs,editor_prop_en,other_prop_en,editor_prop_switches,other_prop_switches,editor_two_quotes,other_two_quotes,editor_en_technical,other_en_technical,editor_score,editor_success
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,True,False,0.341682,0.000000,0.209346,0.000000,True,False,0.065934,0.000000,0.932735,True
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,False,True,0.000000,0.341682,0.000000,0.209346,False,True,0.000000,0.065934,1.000000,True
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,True,False,0.253731,0.000000,0.159091,0.000000,True,False,0.088235,0.000000,0.932735,True
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,False,True,0.000000,0.181818,0.000000,0.113514,False,True,0.000000,0.088235,1.000000,True
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,False,True,0.000000,0.126866,0.000000,0.080153,False,True,0.000000,0.088235,0.205736,False
5,أقباط,مع إحترامي لك لكن,Dukkani,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,Where I get population of copts by Governorate...,2,9,False,True,0.090909,0.008913,0.200000,0.010870,False,True,1.000000,0.166667,0.930295,True
6,أقباط,مع إحترامي لك لكن,Turkmen oglu,حضرتكم تشرحون عن مجموعة دينية كيف لك أن تجعلها...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,2,9,False,True,0.000000,0.011321,0.000000,0.015296,False,True,0.000000,0.285714,1.000000,True
7,أقباط,مع إحترامي لك لكن,أحمد,دراسةhttp://dirkschweitzer.net/E3b-papers/Hass...,Y-chromosomal Aaron\nY-chromosomal Adam\nMitoc...,1,10,False,True,0.000000,0.012270,0.000000,0.016632,False,True,0.000000,0.285714,0.236348,False
8,أمازيغ,أصل كلمة بربري...,Crackall,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",هناك جزيرة في اليمن تدعى بربرة البربري لها علا...,1,3,False,True,0.000000,0.019868,0.000000,0.013699,False,True,0.000000,0.000000,0.231527,False
9,أمازيغ,أصل كلمة بربري...,ولد عيسى,إذا رجعنا إلى اللاتينية ومعنا النظر في الكلمة:...,"و للملاحظة : كلمات سبو(نهر بالمغرب) , سايس(هصب...",1,3,True,False,0.025000,0.000000,0.016949,0.000000,True,False,0.000000,0.000000,0.801391,True


In [29]:
crit = feats['editor_talk'].map(has_english)
n_cs = len(feats[crit])
print("Number of posts with English: {0} / {1}".format(n_cs, len(feats)))

NameError: name 'has_english' is not defined

In [6]:
# Number of threads
len(set(zip(feats['article'], feats['thread_title'])))

48

In [7]:
# Number of editors
len(set(feats['editor']))

73

In [4]:
# See if a text has at least three English words
def has_english(text):
    if sum(w in endict for w in str(text).split() if len(w) > 1) > 2:
        return True
    else:
        return False

# Score editors

In [2]:
# FNS FOR SCORING EDITORS IN REVERT DISCUSSIONS
def dict_diff(orig, chg):
    """ Calculates diff between dictionary a and b based on keys from dict a
        Returns: (edit_score, #tokens changed in edit)
    """
    
    if len(orig) == 0: # no change in original except for stopwords
        return (1.0, 0)
    
    chg_abs_sum = 0 # relevant word changes
    orig_abs_sum = 0
    for k in orig:
        orig_abs_sum += abs(orig[k])
        if orig[k] * chg[k] <= 0: # signs differ
            chg_abs_sum += abs(chg[k])
            
    if orig_abs_sum == 0: 
        if chg_abs_sum == 0: # no sign difference, so revert successful
            return 1.0, abs(sum(orig.values()))
        else:
            debug_here()
            
    return max(1-chg_abs_sum/orig_abs_sum, 0.0), orig_abs_sum

# Test dict_diff
print('reverter loses')
a = {'a': -3, 'b': -5}
b = {'a': 3, 'b': 5}
print(dict_diff(a,b))

print("reverter wins")
a = {'a': -3, 'b': -5}
b = {'a': 0, 'b': 0}
print(dict_diff(a,b))

print("reverter loses badly")
a = {'a': -3, 'b': -5}
b = {'a': 3, 'b': 6}
print(dict_diff(a,b))

print('reverter wins well')
a = {'a': -3, 'b': -5}
b = {'a': -5, 'b': -10}
print(dict_diff(a,b))

print('partial')
a = {'a': -3, 'b': -5, 'c': 4}
b = {'a': 1, 'b': 3, 'c': 6}
print(dict_diff(a,b))

print('reverter gets all changes but one badly')
a = {'a': -3, 'b': -5, 'c': 4}
b = {'a': -1, 'b': -1, 'c': -6}
print(dict_diff(a,b))

reverter loses
(0.0, 8)
reverter wins
(1.0, 8)
reverter loses badly
(0.0, 8)
reverter wins well
(1.0, 8)
partial
(0.6666666666666667, 12)
reverter gets all changes but one badly
(0.5, 12)


In [15]:
# Load, initialize data
# rv_data = pd.read_csv('/home/michael/school/research/wp/revert_talk_threads_unique_7days.csv', parse_dates=['revert_timestamp'])
talk = pd.read_csv('/home/michael/school/research/wp/ar/cs_revert_discussion.csv', parse_dates=['timestamp'])
diff_dir = '/home/michael/school/research/wp/ar/ar_article_diffs/'
assert os.path.exists(diff_dir)
thread_durs = []
talk

,article_title,thread_title,timestamp,username,post_type,post_text
0,ساكب,المصادر,2012-04-30 20:45:34,3Princip,THREAD_STARTER,جميع المصادر التي تنسب العياصرة إلى آل البيت، ...
1,ساكب,المصادر,2012-05-01 01:15:11,3Princip,POST,::ليست محاولات إحباط، ولكن محاولة لتحسين جودة ...
2,ساكب,المصادر,2012-05-01 15:39:22,3Princip,POST,ما رأيك؟ هكذا صارت أحسن؟ أضفت أيضاً آل خليفة ل...
3,سرطان,المصطلحات الانكليزية في المقالة,2008-01-17 11:04:21,A.Obeidat,POST,أرجوا من الزملاء إعطاء رأي بالموضوع--A e t h e...
4,سرطان,المصطلحات الانكليزية في المقالة,2008-01-26 10:41:09,A.Obeidat,POST,يا أخي لا حساسية ولا بطيخ.. الموضوع في موسوعة ...
5,إندورفين,سوء فهم,2009-03-06 21:33:14,Abanima,POST,ما أراه في ويكيبيديا الإنكليزية حالياً لا يؤكد...
6,مشبهة,تحيز,2016-04-08 17:44:38,Aboluay,POST,فمعظم المقالة لا تتحدث عن آراء المشبهة وعقيدته...
7,سرطان,المصطلحات الانكليزية في المقالة,2008-01-17 13:49:00,Ahmedfsalem,POST,3. أن ذكر المصطلح الإنجليزي لا يضر وهو نوع من ...
8,سرطان,المصطلحات الانكليزية في المقالة,2008-01-21 08:31:41,Ahmedfsalem,POST,شباب.... أنا جديد في الويكيبيديا .... ولكن لا ...
9,سرطان,المصطلحات الانكليزية في المقالة,2008-01-21 14:50:15,Ahmedfsalem,POST,--Ahmedfsalem 14:50، 21 يناير 2008 (UTC)


In [5]:
# SCORE THREAD-WISE WINNERS FOR EACH DISCUSSION AND EDIT PARTICIPANT
art_data = pd.DataFrame() # Final spreadsheet with scores, will be organized by article edit keys
# art_data.columns[4] = 'edit_timestamp'
# art_data.rename(columns={'timestamp':'edit_timestamp'}, inplace=True)
art_data['edit_timestamp'] = pd.Series()
art_data['edit_score'] = pd.Series()
art_data['editor_thread_score'] = pd.Series()
art_data['additions'] = pd.Series()
art_data['deletions'] = pd.Series()
art_data['comparison_timestamp'] = pd.Series()

threads = sorted(list(set(zip(talk['article_title'], talk['thread_title']))))

stops = stopwords.words('arabic') 
prev_art = ''

In [46]:
threads

[('أبوكريفون', 'تعليق جديد على الأخ Lavivier'),
 ('أبوكريفون', 'رد على تعليق Lavivier'),
 ('أذربيجان', 'تنبيه: وصلة ميتة تحقّق منها'),
 ('أذربيجان', 'تنبيه: وصلة ميتة تحقّق منها 2'),
 ('أذربيجان', 'تنبيه: وصلة ميتة تحقّق منها 3'),
 ('أذربيجان', 'تنبيه: وصلة ميتة تحقّق منها 4'),
 ('أذربيجان', 'تنبيه: وصلة ميتة تحقّق منها 5'),
 ('أذربيجان', 'تنبيه: وصلة ميتة تحقّق منها 6'),
 ('أذربيجان', 'غير معنون'),
 ('أقباط', 'مع إحترامي لك لكن'),
 ('أمازيغ', 'أصل كلمة بربري...'),
 ('أمازيغ', 'الهند وباكستان R1 و R2'),
 ('إباضية', 'ملاحظة'),
 ('إرهاب', 'TERRORISM'),
 ('إنتروبيا', 'متغيرات الحالة و متغيرات متعلقة بالمسار'),
 ('إندورفين', 'سوء فهم'),
 ('إندونيسيا', 'المصادر'),
 ('الأردن', 'ملاحظة'),
 ('البهنسا', 'el-Bahnasa issue'),
 ('السعودية', 'اقتراح'),
 ('العراق', 'توضيح'),
 ('الفاتيكان', 'غير معنون'),
 ('القمر', 'مقدمة تعريفية أساسية مفقودة في مقالة مختارة؟'),
 ('اللات', 'صحة المعلومات'),
 ('اليمن', 'الرجاء التعديل'),
 ('بغداد', 'اقتراح'),
 ('بيروت', 'غير معنون'),
 ('تركيا', '?السكان'),
 ('تركيا',

In [52]:
# Get edits by thread editors in between initial revert and session end
# session end: end of thread or last revert 7 days after last thread entry by thread participants
art_data = pd.DataFrame()
# for i, (art, thread) in enumerate(threads[:20]):
for i, (art, thread) in enumerate(threads):
    
    if i % 5 == 0:
        print(i)
    
    # Talk page participants--all of them with the corresponding thread in revert data
    talk_parts = set(talk[(talk['article_title']==art) & (talk['thread_title']==thread)]['username'].values)
    
    talk_rows = talk[(talk['article_title'] == art) 
                            & (talk['thread_title'] == thread)
                            ].loc[:, ['article_title', 'thread_title', 'post_timestamp']]
    thread_end = max(talk['timestamp'])
    thread_beg = min(talk['timestamp'])
    
    # Build edit history from beg to end of thread
#     rv_rows = rv_data[(rv_data['article']==art) & (rv_data['thread_title']==art) &
#                      (rv_data['editor'].isin(talk_parts))]
#     initial_rv = min(rv_rows['revert_timestamp'])
    if prev_art != art:
        artfp = os.path.join(diff_dir, art.replace(' ', '_').replace('/', '_') + '_diff.csv')
        if not os.path.exists(artfp):
            debug_here()
        diff_data = pd.read_csv(artfp, parse_dates=['timestamp'])
#         diff_data = pd.read_csv(os.path.join(diff_dir, art.lower().replace(' ', '_').replace('/', '_') + '_diff.csv'),
#                    parse_dates=['timestamp'])
#     sess_beg = min(thread_beg, initial_rv)
#     sess_end = max(thread_end, last_rv)
    sess_beg = thread_beg - DateOffset(days=1)
    sess_end = thread_end + DateOffset(days=1)

    # Find edits that are in same timeframe as thread and which thread participants make
    sess_edits = diff_data.loc[(diff_data['timestamp'] >= sess_beg) & (diff_data['timestamp'] < sess_end)
                         & diff_data['editor'].isin(talk_parts)] # could be intervening edits by non-talk participants
    sess_parts = set(sess_edits['editor'].tolist())
    
    if sess_edits.empty:
        print('No diffs')
        continue
#         # Remove rows from art_data
#         art_data = art_data[(art_data['article'] != art) & (art_data['thread_title'] != thread)] 
    # Recalculate session end and beginning in case talk participants didn't make edits
    sess_beg = min(sess_edits['timestamp'])
    sess_end = max(sess_edits['timestamp'])
    
    edscores = defaultdict(lambda: [0,0]) # [n_wds_successful, n_wds_changed]
    
    # Calculate success score for each edit compared with end revision
    for row in sess_edits.itertuples():
        edit_text = diff_data.loc[diff_data['timestamp']==row.timestamp]
        diffs = diff_data.loc[(diff_data['timestamp'] > row.timestamp) & (diff_data['timestamp'] <= (sess_end + DateOffset(days=1)))]
        
        # Unigram counter for edit
        if (len(edit_text['deletions'].values) > 0) and (isinstance(edit_text['deletions'].values[0], str)):
            positive_dels = Counter([w.lower() for w in edit_text['deletions'].values[0].split() if w.lower() not in stops])
            edit_diff = Counter({key:-1*positive_dels[key] for key in positive_dels})
        else:
            edit_diff = Counter()
        if (len(edit_text['additions'].values) > 0) and (isinstance(edit_text['additions'].values[0], str)):
            adds = Counter([w.lower() for w in edit_text['additions'].values[0].split() if w.lower() not in stops])
        else:
            adds = Counter()
        edit_diff.update(adds)
        edit_diff = {k: edit_diff[k] for k in edit_diff if edit_diff[k] != 0}
        
        if diffs.empty: # No revisions after thread end
            edit_score = 1.0
            n_wds = abs(sum(edit_diff.values()))
            edscores[row.editor][0] += edit_score * n_wds
            edscores[row.editor][1] += n_wds
                
        else: 
            
            # Unigram counter for revision diffs in window
            next_dels = ' '.join(d.lower() for d in diffs['deletions'].values.tolist() if isinstance(d, str))
            changes = Counter([w for w in next_dels.split() if w not in stops])
            changes = Counter({key:-1*changes[key] for key in changes})
            next_adds = ' '.join(a.lower() for a in diffs['additions'].values.tolist() if isinstance(a, str))
            next_addwds = Counter([w for w in next_adds.split() if w not in stops])

            changes.update(next_addwds)
            
            edit_score, n_wds = dict_diff(edit_diff, changes)
            edscores[row.editor][0] += edit_score * n_wds
            edscores[row.editor][1] += n_wds

        # Add score to dataframe (or row if the edit wasn't a revert)
#         match = art_data[(art_data['article']==row.article_name) & 
#                          (art_data['thread_title']==art) &
#                         (art_data['editor']==row.editor) &
#                         (art_data['edit_timestamp']==row.timestamp)]

#         if not match.empty:
#             art_data.set_value(match.index[0], 'edit_score', edit_score)
#             art_data.loc[match.index[0], 'additions'] = row.additions
#             art_data.loc[match.index[0], 'deletions'] = row.deletions
#             if not diffs.empty: 
#                 art_data.loc[match.index[0], 'comparison_timestamp'] = max(diffs['timestamp'])
# #             art_data.set_value(match.index[0], 'winner', winner)
#         else:
        new_row = pd.DataFrame([row])
        new_row['edit_score'] = edit_score
#             new_row['winner'] = winner
        new_row['thread_title'] = thread
        if not diffs.empty:
            new_row['comparison_timestamp'] = max(diffs['timestamp'])
        new_row.rename(columns={'timestamp': 'edit_timestamp', 'article_name': 'article'}, inplace=True)
        new_row.drop('Index', axis=1, inplace=True)
        art_data = art_data.append(new_row, ignore_index=True)
            
    art_data.reset_index(drop=True)
    prev_art = art
    
    # Calculate editor thread scores
    for ed in sess_parts:
        sess_finalrows = art_data[(art_data['article']==art) & 
                        (art_data['thread_title']==thread) &
                        (art_data['editor']==ed)]
        if edscores[ed][1] == 0: # editor whose only edit was the final one and was of no words
            ed_threadscore = 1.0
        else:
            ed_threadscore = edscores[ed][0]/edscores[ed][1]
        if ed_threadscore > 1.0 or ed_threadscore < 0.0:
            debug_here()
        for idx in sess_finalrows.index:
            art_data.loc[idx, 'editor_thread_score'] = ed_threadscore
    
# Sort art_data
art_data.sort_values(['article', 'thread_title', 'edit_timestamp'], inplace=True)

# Select columns
cols = ['article', 'thread_title', 'edit_timestamp', 'editor', 'edit_comment',
       'edit_score', 'editor_thread_score', 'comparison_timestamp', 'additions', 'deletions']
art_data = art_data[cols]

# Remove reverts that don't have diffs (just removed Wikipedia metadata, for instance)
mask = [isinstance(tup[0], str) or isinstance(tup[1], str) for tup in zip(art_data['additions'], art_data['deletions'])]
art_data = art_data[mask]
    
art_data.drop_duplicates(inplace=True)
art_data.to_csv('/home/michael/school/research/wp/ar/editor_thread_scores.csv', index=False)
len(art_data)

0
5
10
15
20
25
30
35
No diffs
40
45
50
55


5038

In [53]:
art_data

,article,thread_title,edit_timestamp,editor,edit_comment,edit_score,editor_thread_score,comparison_timestamp,additions,deletions
0,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-04-25 10:30:17,Aram33,NaN,0.862069,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة تمكنوا الحصول بأ...,قيامه. العام. العضوي (معرفة) غنوسيس
1,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-04 23:42:13,Aram33,الرجوع عن التعديل 1675394 بواسطة [[خاص:مساهمات...,0.862069,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة تمكنوا الحصول بأ...,قيامه. العام. العضوي (معرفة) غنوسيس
2,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-06 20:36:48,Aram33,للأسف يتم تحويل المقال لمقالة بوليسية عن أسرار...,0.950000,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة تمكنوا تعتقد الح...,أَمَّا يَفْهَمُونَ، يَتُوبُوا سِرَّ للعهد قيام...
3,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-18 22:03:49,Aram33,وجود مقال ما في ويكيبييا الإنكليزية لا تعني بأ...,0.885714,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة تمكنوا تعتقد الح...,قيامه. العام. (معرفة) للعهد غنوسيس في القانوني...
4,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-21 10:53:38,Aram33,الرجوع عن التعديل 1713459 بواسطة [[خاص:مساهمات...,0.885714,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة تمكنوا تعتقد الح...,قيامه. العام. (معرفة) للعهد غنوسيس في القانوني...
5,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-24 09:58:04,Aram33,NaN,0.923077,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة تمكنوا تعتقد الح...,قيامه. العام. (معرفة) للعهد غنوسيس في القانوني...
6,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-24 15:07:19,Aram33,هل تريد المزيد من المصادر التي تبين أن الغنوصي...,0.854167,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة المسيحية تعتقد ا...,قيامه. العام. (معرفة) للعهد غنوسيس في القانوني...
7,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-24 15:45:01,Aram33,هل تمزح معي؟,0.854167,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة المسيحية تعتقد ا...,قيامه. العام. (معرفة) للعهد غنوسيس في القانوني...
8,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-24 16:52:16,Aram33,ربما سأوقف الآن عن التحرير، إن لم تقنعك إجابتي...,0.854167,0.932735,2011-09-02 07:09:17,غنوصية لنخبة فالغنوصية بإنارة المسيحية تعتقد ا...,قيامه. العام. (معرفة) للعهد غنوسيس في القانوني...
9,أبوكريفون,تعليق جديد على الأخ Lavivier,2008-05-26 20:40:30,Histolo2,NaN,1.000000,1.000000,2011-09-02 07:09:17,NaN,أَمَّا يَفْهَمُونَ، يَتُوبُوا سِرَّ فَتُغْفَرَ...


# Build features for thread posts and editors

In [56]:
len(threads)

59

## Thread post text

In [55]:
# Build input corpora of editors' text + others' text, get labels
# poss_edthreads = set(zip(talk['article_title'], talk['thread_title'], talk['username']))
edthreads = sorted(set((zip(art_data['article'], art_data['thread_title'], art_data['editor'], art_data['editor_thread_score']))))
edthreads

# edthreads = list()
# scores = {line[:3]: line[3] for line in edthreads}

# for i, el in enumerate(sorted(edthreads)):
#     rows = art_data[(art_data['article']==el[0]) &
#                     (art_data['thread_title']==el[1]) &
#                     (art_data['editor']==el[2])]
#     if not rows.empty:
# #         edthreads.append(el)
#         score = rows['editor_thread_score'].iloc[0]
#         scores[el] = score

# Assemble input text of just editors' text
edtalk = defaultdict(str)
n_edturns = {}
n_otherturns = {}
othertalk = defaultdict(str)

for i, el in enumerate(edthreads):
    rows = talk[(talk['article_title']==el[0]) &
                    (talk['thread_title']==el[1])]
    edrows = rows[rows['username']==el[2]]
    edtalk[el] += ' '.join([str(t) for t in edrows['post_text'].tolist()])
    n_edturns[el] = len(edrows)
    
    other_rows = rows[rows['username']!=el[2]]
    othertalk[el] += ' '.join([str(t) for t in other_rows['post_text'].tolist()])
    n_otherturns[el] = len(other_rows)
    
# Build one relevant dataframe
outrows = []
for i, el in enumerate(edthreads):
    outrows.append([el[0], el[1], el[2], edtalk[el], othertalk[el], n_edturns[el], n_otherturns[el],
                    el[3]])
    
talk_scores = pd.DataFrame(outrows, columns=['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
                              '#editor_turns', '#other_turns', 'editor_score'])
talk_scores.to_csv('/home/michael/school/research/wp/ar/cs_discussion_features.csv', index=False)
talk_scores

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score
0,أبوكريفون,تعليق جديد على الأخ Lavivier,Aram33,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,4,4,0.932735
1,أبوكريفون,تعليق جديد على الأخ Lavivier,Histolo2,تحية طيبة مرة أخرى صديقي العزيز، وأود أن أستغل...,لا يهم إن وجد 3 أو 5000 نسخة لقانون الإيمان ال...,4,4,1.000000
2,أبوكريفون,رد على تعليق Lavivier,Aram33,The main differences with Christianity and gno...,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,2,3,0.932735
3,أبوكريفون,رد على تعليق Lavivier,Histolo2,مع حترامي الشديد لك أخي عزيزي ولكنك تخوض في مو...,The main differences with Christianity and gno...,2,3,1.000000
4,أبوكريفون,رد على تعليق Lavivier,Lavivier,كيف تعلم إني لم أقرأ مصادرك أو لم أسمع عن العق...,The main differences with Christianity and gno...,1,4,0.205736
5,أذربيجان,تنبيه: وصلة ميتة تحقّق منها,باسم,بعد عمليات آلية عدة، كُشِف أن الوصلة الخارجية ...,,1,0,0.791159
6,أذربيجان,تنبيه: وصلة ميتة تحقّق منها 2,باسم,بعد عمليات آلية عدة، كُشِف أن الوصلة الخارجية ...,,1,0,0.791159
7,أذربيجان,تنبيه: وصلة ميتة تحقّق منها 3,باسم,بعد عمليات آلية عدة، كُشِف أن الوصلة الخارجية ...,,1,0,0.791159
8,أذربيجان,تنبيه: وصلة ميتة تحقّق منها 4,باسم,بعد عمليات آلية عدة، كُشِف أن الوصلة الخارجية ...,,1,0,0.791159
9,أذربيجان,تنبيه: وصلة ميتة تحقّق منها 5,باسم,بعد عمليات آلية عدة، كُشِف أن الوصلة الخارجية ...,,1,0,0.791159


In [57]:
# Remove singletons
threads = list(zip(talk_scores['article'], talk_scores['thread_title']))
singles = [t for t in threads if threads.count(t) == 1]
len(singles)

27

In [59]:
mask = [True]*len(talk_scores)
for i, line in talk_scores.iterrows():
    if (line['article'], line['thread_title']) in singles:
        mask[i] = False
        
talk_scores_nosingles = talk_scores[mask]
len(talk_scores_nosingles)
talk_scores_nosingles.to_csv('/home/michael/school/research/wp/ar/cs_discussion_features.csv', index=False)

# Remove duplicates from cs revert conversations

In [3]:
cs_revert_threads = pd.read_csv('/home/michael/school/research/wp/ar/cs_revert_discussion.csv')
cs_revert_threads.drop_duplicates(inplace=True)
len(cs_revert_threads)

293

In [4]:
cs_revert_threads.to_csv('/home/michael/school/research/wp/ar/cs_revert_discussion.csv', index=False)

# Check for cs conversations with reverts

In [11]:
# Load talk data
cs_threads = pd.read_csv('/home/michael/school/research/wp/ar/artalk_cs.csv', parse_dates=['timestamp'])
cs_threads

,article_title,thread_title,timestamp,username,post_type,post_text
0,حرب النجوم (فيلم),المقدمة,2014-04-06 22:09:29,-XQV-,THREAD_STARTER,دائما في كل جزء من الفليم له مقدمة بنفس الطريق...
1,Discussion:ليوناردو دي كابريو,جملة,2014-08-31 07:31:31,1999fran,THREAD_STARTER,مرحبأ بألزملاء ما هي أفضل ترجمة (haloed by fir...
2,محمود درويش,date of birth,2012-03-14 16:11:39,3Princip,POST,شكراً لملاحظتك الدقيقة. الآن المعلومة معدلة حس...
3,Discussion:محمود درويش,date of birth,2012-03-14 16:11:39,3Princip,POST,شكراً لملاحظتك الدقيقة. الآن المعلومة معدلة حس...
4,ساكب,المصادر,2012-04-30 20:45:34,3Princip,THREAD_STARTER,جميع المصادر التي تنسب العياصرة إلى آل البيت، ...
5,ساكب,المصادر,2012-05-01 01:15:11,3Princip,POST,::ليست محاولات إحباط، ولكن محاولة لتحسين جودة ...
6,ساكب,المصادر,2012-05-01 15:39:22,3Princip,POST,ما رأيك؟ هكذا صارت أحسن؟ أضفت أيضاً آل خليفة ل...
7,Discussion:لغة ألمانية,هل يشترط وضع روابط مفتوحة المصدر أم لا؟,2009-09-19 03:56:59,A.b.m.k,POST,vc x cxsx feajpitgfpaewgj ohgiwefhnpaiwejft hj...
8,سرطان,المصطلحات الانكليزية في المقالة,2008-01-17 11:04:21,A.Obeidat,POST,أرجوا من الزملاء إعطاء رأي بالموضوع--A e t h e...
9,سرطان,المصطلحات الانكليزية في المقالة,2008-01-26 10:41:09,A.Obeidat,POST,يا أخي لا حساسية ولا بطيخ.. الموضوع في موسوعة ...


In [3]:
# Organize into threads
threadtimes = {} # (article_title, thread_title): [beg_timestamp, end_timestamp, [users]]

threads = set(zip(cs_threads['article_title'], cs_threads['thread_title']))
len(threads)

1274

In [4]:
# Collect thread info
for art_name, t in threads:
    rows = cs_threads[(cs_threads['article_title']==art_name) & (cs_threads['thread_title']==t)]
    beg = min(rows['timestamp'])
    end = max(rows['timestamp'])
    users = set(rows['username'].tolist())
    threadtimes[(art_name, t)] = [beg, end, users]

%xdel cs_threads
len(threadtimes)

1274

In [9]:
# Number of reverts within cs discussion timeframes 
rev_dirpath = '/home/michael/school/research/wp/ar/ar_articles/with_reverts'
n_matches = 0
n_posts = 0
match_threads = []

for i, (art_name, t) in enumerate(threadtimes):
#     print(art_name)
    
    beg = threadtimes[art_name, t][0] - DateOffset(days=7)
    end = threadtimes[art_name, t][1] + DateOffset(days=7)
    users = threadtimes[art_name, t][2]
    
    if art_name.startswith('Discussion:'):
        art_name = art_name[len('Discussion:'):]
    
    # Load revisions
    rev_path = os.path.join(rev_dirpath, art_name + '.csv')
    if os.path.isfile(rev_path):
        print(n_matches, end="/")
        print(i)
#         print(i, end="\t")
#         print(n_posts)
        chunks = pd.read_csv(rev_path, parse_dates=[1], header=None, iterator=True, chunksize=1000)
        revs = pd.concat(chunks, ignore_index=True)
        %xdel chunks
        reverts = revs[revs[5] == 1]
    
        time_mask = [ts >= beg and ts <= end for ts in reverts[1]]
        selected_rows = reverts[time_mask]
        if not selected_rows.empty:
            selected_rows = selected_rows[selected_rows[3].isin(users)]
        if not selected_rows.empty:
            # Check if reverted also in discussion
            sel_posts = 0 # actually are article edits, not posts
            for ind in selected_rows.index: 
                if revs.loc[ind-1, 3] in users:
                    sel_posts += 1
            n_posts += sel_posts
            if sel_posts > 0:
                n_matches += 1
                match_threads.append((art_name, t))
                
        %xdel revs

n_matches

0/2
0/5
1/7
1/10
2/11
3/12
3/13
3/21
3/22
3/23
3/26
3/28
4/31
4/32
4/33
4/37
4/50
4/51
5/58
5/61
6/62
6/63
7/74
7/79
7/83
7/87
8/91
8/93
9/104
9/107
9/108
9/111
9/116
9/119
9/125
9/126
9/127
10/131
10/136
10/139
10/146
10/150
11/155
11/156
11/159
11/163
11/169
12/174
12/175
12/176
12/179
12/180
12/181
12/184
13/185
14/186
14/189
14/196
14/201
14/204
14/206
15/212
15/214
15/215
15/216
15/219
15/222
16/229
17/233
18/235
18/238
19/240
20/247
20/253
20/259
20/260
21/262
21/266
22/267
22/269
23/276
24/277
24/287
25/297
25/299
26/305
26/308
27/309
27/312
28/313
29/321
29/324
30/325
30/329
30/330
31/333
31/334
31/344
31/345
32/350
32/353
32/356
32/360
32/362
32/363
33/364
34/371
34/375
34/382
34/385
34/389
34/390
35/393
35/396
35/399
35/400
35/403
35/404
35/409
35/410
35/413
36/414
37/416
38/417
38/419
38/422
38/426
38/427
38/428
38/440
39/443
40/445
40/448
40/450
40/453
40/454
40/458
40/460
40/463
40/464
40/465
41/466
42/470
42/473
43/475
43/479
44/486
44/487
44/489
44/490
45/493
46/500
47/5

107

In [12]:
# Make csv of discussion posts
rev_threads_inds = []
for art_name, t in match_threads:
    rows = cs_threads[(cs_threads['article_title']==art_name) & (cs_threads['thread_title']==t)]
    rev_threads_inds.extend(rows.index)

In [17]:
# Will want to remove duplicates, which come from the original cs_threads having 'Discussion' or not
outpath = '/home/michael/school/research/wp/ar/cs_revert_discussion.csv'
cs_revert_talk = cs_threads.loc[rev_threads_inds, :].sort_index()
cs_revert_talk

,article_title,thread_title,timestamp,username,post_type,post_text
4,ساكب,المصادر,2012-04-30 20:45:34,3Princip,THREAD_STARTER,جميع المصادر التي تنسب العياصرة إلى آل البيت، ...
5,ساكب,المصادر,2012-05-01 01:15:11,3Princip,POST,::ليست محاولات إحباط، ولكن محاولة لتحسين جودة ...
6,ساكب,المصادر,2012-05-01 15:39:22,3Princip,POST,ما رأيك؟ هكذا صارت أحسن؟ أضفت أيضاً آل خليفة ل...
8,سرطان,المصطلحات الانكليزية في المقالة,2008-01-17 11:04:21,A.Obeidat,POST,أرجوا من الزملاء إعطاء رأي بالموضوع--A e t h e...
8,سرطان,المصطلحات الانكليزية في المقالة,2008-01-17 11:04:21,A.Obeidat,POST,أرجوا من الزملاء إعطاء رأي بالموضوع--A e t h e...
9,سرطان,المصطلحات الانكليزية في المقالة,2008-01-26 10:41:09,A.Obeidat,POST,يا أخي لا حساسية ولا بطيخ.. الموضوع في موسوعة ...
9,سرطان,المصطلحات الانكليزية في المقالة,2008-01-26 10:41:09,A.Obeidat,POST,يا أخي لا حساسية ولا بطيخ.. الموضوع في موسوعة ...
50,إندورفين,سوء فهم,2009-03-06 21:33:14,Abanima,POST,ما أراه في ويكيبيديا الإنكليزية حالياً لا يؤكد...
50,إندورفين,سوء فهم,2009-03-06 21:33:14,Abanima,POST,ما أراه في ويكيبيديا الإنكليزية حالياً لا يؤكد...
92,مشبهة,تحيز,2016-04-08 17:44:38,Aboluay,POST,فمعظم المقالة لا تتحدث عن آراء المشبهة وعقيدته...


In [18]:
cs_revert_talk.to_csv(outpath, index=None)

# See which revisions are reverts

In [29]:
fnames = []
dirpath = '/home/michael/school/research/wp/wp_articles/extract/extract_articles/ar_articles'
for fname in os.listdir(dirpath):
    if fname.endswith('.tsv'):
        revpath = os.path.join(dirpath, fname)
        reverts = []
    #     revs = pd.read_csv(revpath, delimiter='\t', header=None)
    #     revs = pd.read_csv(revpath, header=None)
        with open(revpath) as f:
            r = csv.reader(f) # be careful of delimiter
            revs = [row for row in r]

        fnames.append(fname)

        hashes = set()
        for rev in revs:
            m = hashlib.md5()
            m.update(str(rev[2]).encode('utf8'))

            #restores previous version
            rv = (1 if m.digest() in hashes else 0)

            if not rv:
                hashes.add(m.digest())

            reverts.append(rv)    

        print(reverts.count(1))

    #     revs[5] = pd.Series(reverts)
        revs = [revs[i] + [reverts[i]] for i in range(len(revs))]

        with open(revpath[:-4] + '.csv', 'w') as f:
            w = csv.writer(f)
            w.writerows(revs)
        
#     revs.to_csv(, index=None, header=None)

71
127


# Write names of Arabic talk pages with threads that have cs

In [11]:
cs_threads = pd.read_csv('/home/michael/school/research/wp/ar/artalk_cs.csv')
cs_threads

cs_articles = set(cs_threads['article_title'].values)
new_cs_articles = set()
for name in cs_articles:
    if name.startswith('Discussion:'):
        newname = name[len('Discussion:'):]
        new_cs_articles.add(newname)
    else:
        new_cs_articles.add(name)

with open('/home/michael/school/research/wp/ar/ar_cs_artnames.txt', 'w') as f:
    for art in new_cs_articles:
        f.write(art + '\n')

# Modify pagenames file

In [3]:
namedata = pd.read_csv('/home/michael/school/research/wp/ar/artalk_pagenames.csv', header=None)
namedata

,0
0,عقوبة الإعدام
1,Discussion:السلام عليك يا مريم
2,Discussion:قومية عربية
3,كولوسيوم
4,متلازمة توريت
5,الثور الهائج
6,اقتصاد إسلامي
7,جائزة الأوسكار لأفضل ممثل مساعد
8,كاتب ياسين
9,هاري بوتر وكأس النار (فيلم)


In [10]:
outfile = '/home/michael/school/research/wp/ar/artalk_pagenames.txt'
with open(outfile, 'w') as f:
    for name in namedata[0].values.tolist():
        f.write(name+'\n')

# Cs stats

In [73]:
talkpath = '/home/michael/school/research/wp/ar/artalk.tsv'
talk = pd.read_csv(talkpath, delimiter='\t')
len(talk)

# talk = pd.read_csv('/home/michael/school/research/wp/ar/artalk.csv')
# len(talk)

22870

In [5]:
with open('/home/michael/school/research/wp/ar/french_utf8.txt') as f:
    frdict = set(f.read().splitlines())
len(frdict)

336531

In [74]:
len(endict)

235892

## Have any ASCII letters

In [7]:
crit = talk['post_text'].map(lambda x: any(c in str(x) for c in ascii_letters))
# crit = talk['discussion_text'].map(lambda x: any(c in str(x) for c in ascii_letters))
some_ascii = talk[crit]
# print("Number of talk pages with ASCII letters: {:d}/{:d} {:.1%}".format(len(some_ascii), len(talk), len(some_ascii)/len(talk)))
print("Number of contributions with ASCII letters: {:d}/{:d} {:.1%}".format(len(some_ascii), len(talk), len(some_ascii)/len(talk)))

Number of talk pages with ASCII letters: 17639/22870 77.1%


## Standalone English dictionary words present (no URLs or names, hopefully)

In [29]:
# crit = talk['discussion_text'].map(lambda x: any(w in endict for w in str(x).split()))
crit = talk['post_text'].map(lambda x: any(w in endict for w in str(x).split()))
has_enwd = talk[crit]
print("Number of posts with at least one English word: {:d}/{:d} {:.1%}".format(len(has_enwd), len(talk), len(has_enwd)/len(talk)))

Number of talk pages with at least one English word: 5241/87125 6.0%


In [75]:
# How many threads have at least one post w cs?
threads = list(set(talk['thread_title'].tolist()))
print(len(threads))

6029


In [76]:
cs_threads = []
for t in threads:
    thread_posts = talk[talk['thread_title']==t]
    crit = thread_posts['post_text'].map(lambda x: sum(w in endict for w in str(x).split() if len(w) > 1) > 2)
    thread_cs = thread_posts[crit]
    if len(thread_cs) > 0:
        cs_threads.append(t)
    
print("Number of threads with a post of at least three English words: {:d}/{:d} {:.1%}".format(len(cs_threads), len(threads), len(cs_threads)/len(threads)))
threads_cs = talk[talk['thread_title'].isin(cs_threads)]
print(len(threads_cs))

cspath = '/home/michael/school/research/wp/ar/artalk_cs.csv'
threads_cs.to_csv(cspath, index=False)

Number of threads with a post of at least three English words: 376/6029 6.2%
3520


In [8]:
# How many posts have at least three English words
crit = talk['post_text'].map(lambda x: sum(w in endict for w in str(x).split() if len(w) > 1) > 2)
# crit = talk['discussion_text'].map(lambda x: sum(w in endict for w in str(x).split() if len(w) > 1) > 2)
enwd3 = talk[crit]
print("Number of posts with at least three English words: {:d}/{:d} {:.1%}".format(len(enwd3), len(talk), len(enwd3)/len(talk)))

Number of posts with at least three English words: 787/22870 3.4%


In [31]:
# How many English words are present in talk pages that have them?
en_counter = Counter()
for i, text in has_enwd['discussion_text'].iteritems():
    n_enwds = sum(w in endict for w in str(text).split())
    en_counter[n_enwds] += 1
    
en_counter 

Counter({1: 1512,
         2: 818,
         3: 518,
         4: 277,
         5: 219,
         6: 184,
         7: 97,
         8: 125,
         9: 120,
         10: 87,
         11: 64,
         12: 70,
         13: 53,
         14: 61,
         15: 30,
         16: 38,
         17: 29,
         18: 31,
         19: 37,
         20: 47,
         21: 33,
         22: 27,
         23: 23,
         24: 23,
         25: 29,
         26: 30,
         27: 19,
         28: 5,
         29: 18,
         30: 21,
         31: 9,
         32: 12,
         33: 12,
         34: 14,
         35: 19,
         36: 20,
         37: 10,
         38: 10,
         39: 14,
         40: 10,
         41: 4,
         42: 10,
         43: 8,
         44: 12,
         45: 8,
         46: 6,
         47: 9,
         48: 8,
         49: 6,
         50: 6,
         51: 10,
         52: 10,
         53: 5,
         54: 16,
         55: 9,
         57: 8,
         58: 4,
         59: 2,
         61: 4,
         62: 

In [32]:
# How many posts/talk pages have more than 2 English words?
en_more = sum(en_counter[v] for v in en_counter if v>2)
print("Number of talk pages with at least three English words: {:d}/{:d} {:.1%}".format(en_more, len(talk), en_more/len(talk)))

Number of talk pages with at least three English words: 2911/87125 3.3%


In [33]:
# In pages that have only one English word, what is it?
singleton_counter = Counter()
for i, text in has_enwd['discussion_text'].iteritems():
    enwds = [w for w in str(text).split() if w in endict]
    if len(enwds) == 1:
        singleton_counter[enwds[0]] += 1
    
singleton_counter.most_common()

[('M', 50),
 ('of', 46),
 ('Arabic', 40),
 ('F', 37),
 ('A', 33),
 ('Ahmed', 22),
 ('Anubis', 22),
 ('el', 21),
 ('Jo', 19),
 ('Alexander', 15),
 ('de', 14),
 ('The', 14),
 ('I', 12),
 ('user', 12),
 ('Abu', 10),
 ('boy', 10),
 ('Arab', 10),
 ('Matthew', 10),
 ('mode', 9),
 ('May', 8),
 ('G', 8),
 ('the', 8),
 ('c', 8),
 ('x', 8),
 ('Mohammad', 8),
 ('U', 8),
 ('H', 8),
 ('i', 8),
 ('n', 8),
 ('L', 6),
 ('pour', 6),
 ('good', 6),
 ('shaker', 6),
 ('C', 6),
 ('en', 6),
 ('or', 6),
 ('love', 6),
 ('number', 6),
 ('g', 6),
 ('Z', 6),
 ('E', 6),
 ('Sara', 6),
 ('teaching', 6),
 ('lost', 5),
 ('on', 4),
 ('al', 4),
 ('K', 4),
 ('Ben', 4),
 ('chaos', 4),
 ('King', 4),
 ('J', 4),
 ('German', 4),
 ('in', 4),
 ('cycle', 4),
 ('B', 4),
 ('ref', 4),
 ('e', 4),
 ('to', 4),
 ('goal', 4),
 ('pointer', 4),
 ('V', 4),
 ('ontology', 4),
 ('Culex', 4),
 ('ana', 4),
 ('section', 4),
 ('rad', 4),
 ('black', 4),
 ('Mac', 4),
 ('bin', 4),
 ('Egyptian', 4),
 ('king', 4),
 ('David', 4),
 ('ma', 4),
 ('amir', 

In [37]:
# Write out CSV of talk pages with at least 3 English words
crit = talk['discussion_text'].map(lambda x: sum(w in endict for w in str(x).split() if len(w) > 1) > 2)
# crit = talk['discussion_text'].map(lambda x: ascii_letters[0] in str(x))
enwd3 = talk[crit]
print("Number of talk pages with at least three English words: {:d}/{:d} {:.1%}".format(len(enwd3), len(talk), len(enwd3)/len(talk)))

Number of talk pages with at least three English words: 2484/87125 2.9%


In [39]:
enwd3.to_csv('/home/michael/school/research/wp/ar/artalk3.csv', index=False)

# French words

In [10]:
# Talk pages with at least 3 French words
crit = talk['discussion_text'].map(lambda x: sum(w in frdict for w in str(x).split() if len(w) > 1) > 2)
# crit = talk['discussion_text'].map(lambda x: ascii_letters[0] in str(x))
frwd3 = talk[crit]
print("Number of talk pages with at least three French words: {:d}/{:d} {:.1%}".format(len(frwd3), len(talk), len(frwd3)/len(talk)))

Number of talk pages with at least three French words: 1362/87125 1.6%
